In [11]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

In [12]:
searchString = input('What do you want to search? ')
flipkart_url = "https://www.flipkart.com/search?q=" + searchString
req = requests.get(flipkart_url)
content = req.text
soup = BeautifulSoup(content,'html.parser')

bigboxes = soup.findAll("div", {"class": "bhgxx2 col-12-12"})

#bigboxes[2].div.div.div.a['href']

What do you want to search? samsung a10


In [13]:
flip = "https://www.flipkart.com"
rating = []
name = []
heading =[]
review = []
for num in range(2,13):
    newUrl = flip + bigboxes[num].div.div.div.a['href']

    req = requests.get(newUrl)
    soup = BeautifulSoup(req.content,'html.parser')
    all = soup.findAll("div",{'class':"col _39LH-M"})


    links = []
    for link in all[0].findAll('a', attrs={'href': re.compile(r"FLIPKART$")}):
            links.append(link.get('href'))


    if(len(links)==0):
        continue
    page_number = 1
    reviewUrl =flip+links[0]+'&page='+str(page_number)
    
    req = requests.get(reviewUrl)
    soup = BeautifulSoup(req.content,'html.parser')
    full = soup.findAll('div',{'class':'ooJZfD _2oZ8XT col-9-12'})

    all = full[0].findAll('div',{'class':'_3gijNv col-12-12'})

    last_page = all[-1].find('div',{'class':'_2zg3yZ _3KSYCY'}).span.text.split(" ")[-1]
    last_page = int(last_page)


    for page in range(page_number,last_page-1):
        reviewUrl =flip+links[0]+'&page='+str(page)
        req = requests.get(reviewUrl)
        #print(reviewUrl)
        soup = BeautifulSoup(req.content,'html.parser')
        full = soup.findAll('div',{'class':'ooJZfD _2oZ8XT col-9-12'})

        try:
            all = full[0].findAll('div',{'class':'_3gijNv col-12-12'})
        except:
            pass
        
        for row in all[2:11]:
            data = row.findAll('div',{'class':'row'})
            try:
                rating.append(data[0].text[0])
            except:
                rating.append('No rating')
            try:
                name.append(data[2].text.split('Certified')[0])
            except:
                name.append('No Name')
            try:
                heading.append(data[0].text[1:])
            except:
                heading.append('No Heading')
            try:
                review.append(data[1].text[:-10])
            except:
                review.append('No review')

In [14]:
product = [searchString]*len(rating)

data = [product,rating,name,heading,review]

df = pd.DataFrame(list(zip(product,rating,name,heading,review)),columns = ['Product','Rating','Name','Heading','Review'])

df.head()

,Product,Rating,Name,Heading,Review
0,samsung a10,5,amith gunaga,Fabulous!,Good product from Samsung...worthy and nice ca...
1,samsung a10,5,DIPAYAN BISWAS,Fabulous!,Baught this as a gift for father. Happiest Fat...
2,samsung a10,5,Hansraj Kakade,Excellent,"Awesome phone , surprise for my mom she loved i"
3,samsung a10,5,Rishabh Pandey,Brilliant,good qualit
4,samsung a10,5,R.K Dhiman,Super!,very good mobil


In [15]:
df.shape

(7659, 5)